In [1]:
# coding: UTF-8
import os
import sys
from pathlib import Path
os.chdir('../')
PROJECT_DIR = Path().resolve().parents[1]
sys.path.append(os.path.join(PROJECT_DIR, 'src'))

In [2]:
# ========== python ==========
from logging import Logger
# noinspection PyUnresolvedReferences
from typing import List, Dict, Tuple, Optional, Union, Callable
# noinspection PyUnresolvedReferences
from tqdm import tqdm
from argparse import Namespace
# Machine learning
import h5py
import numpy as np
import pandas as pd
import optuna
# torch
import torch
from torch import nn
from torch.utils.data.dataloader import DataLoader

from models.run import *

In [3]:
args, logger, device = setup(setup_parser, PROJECT_DIR, parser_args = [
    '--logfile', 'log/test.log', '--console-level', 'debug', '--param-file', 'log/test.pkl', '--KGdata',
    'WN18RR', '--console-level', 'debug', '--function', 'do_1train',
    '--eco-memory', '--model', 'transformere2',
    '--entity-special-num', '3', '--relation-special-num', '3',
    '--padding-token-e', '0', '--cls-token-e', '1',
    '--padding-token-r', '0', '--cls-token-r', '1', '--self-loop-token-r', '2',
    '--batch-size', '4',
    '--do-debug'
])

In [4]:
progress_helper = ProgressHelper("log/progress.{pid}.txt", pid=args.pid)

In [5]:
args.project_dir = PROJECT_DIR
args.logger = logger
args.device = device
args.completed = {}
const_args = ChangeDisableNamespace(args)

In [6]:
data_helper, model = do_1train(const_args, logger=logger, progress_helper=progress_helper)

2022-09-25 15:44:27 - INFO - run.py - 394 - =============Function start=============
2022-09-25 15:44:27 - INFO - run.py - 359 - =======load data start.         ========
2022-09-25 15:44:27 - DEBUG - data_helper.py - 74 - {0: '_hypernym', 1: '_derivationally_related_form', 2: '_instance_hypernym', 3: '_also_see', 4: '_member_meronym', 5: '_synset_domain_topic_of', 6: '_has_part', 7: '_member_of_domain_usage', 8: '_member_of_domain_region', 9: '_verb_group', 10: '_similar_to', 11: '_hypernym_REVERSE', 12: '_derivationally_related_form_REVERSE', 13: '_instance_hypernym_REVERSE', 14: '_also_see_REVERSE', 15: '_member_meronym_REVERSE', 16: '_synset_domain_topic_of_REVERSE', 17: '_has_part_REVERSE', 18: '_member_of_domain_usage_REVERSE', 19: '_member_of_domain_region_REVERSE', 20: '_verb_group_REVERSE', 21: '_similar_to_REVERSE'}
2022-09-25 15:44:28 - INFO - run.py - 361 - =====this is  eco_memory mode=====
2022-09-25 15:44:28 - INFO - run.py - 362 - =======load data complete.      ========

In [7]:
train = data_helper.train_dataloader

In [71]:
er, e2s = None, None
for er, e2s in train:
    er, e2s = er, e2s
    e, r = er.split(1, 1)
    break
er, e2s

(tensor([[30959,     3],
         [39938,     3],
         [37244,    15],
         [  326,     3]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [78]:
import importlib
import model as model_file
importlib.reload(model_file)
TransformerVer3E = model_file.TransformerVer3E
model_ = TransformerVer3E(
    args, data_helper.get_final_e_length(), data_helper.get_final_r_length(),
    data_helper=data_helper, logger=logger)

2022-09-26 12:20:42 - DEBUG - pytorch_geometric.py - 150 - e_length: 40943,	 graph_node_num: 40946
2022-09-26 12:20:42 - DEBUG - pytorch_geometric.py - 151 - r_length: 22,	 graph_edge_num: 25
2022-09-26 12:20:44 - INFO - pytorch_geometric.py - 181 - is_directed: True
2022-09-26 12:21:27 - DEBUG - pytorch_geometric.py - 195 - node2neighbor_node shape: torch.Size([40946, 483])


In [79]:
model_.forward((e, r))

tensor([[5.0000e-01, 7.0592e-06, 7.0720e-01,  ..., 7.9204e-01, 5.7981e-02,
         9.7943e-01],
        [5.0000e-01, 9.2120e-06, 3.0770e-01,  ..., 9.3850e-01, 6.5375e-01,
         8.9034e-01],
        [5.0000e-01, 6.7717e-06, 9.7483e-01,  ..., 9.5228e-01, 1.3007e-03,
         9.8940e-01],
        [5.0000e-01, 3.2054e-03, 9.7117e-01,  ..., 8.7392e-02, 6.2707e-01,
         9.9734e-01]], grad_fn=<SigmoidBackward0>)

In [62]:
batch_size = e.shape[0]
neighbor_node = model_.node2neighbor_node[e].view(batch_size, -1)
neighbor_attr = model_.node2neighbor_attr[e].view(batch_size, -1)
neighbor_pad = model_.node2neighbor_pad[e].view(batch_size, -1)

print(r[0])
print(neighbor_attr[0])

neighbor_node[neighbor_node==r] = 0
neighbor_attr[neighbor_attr==r] = 0
neighbor_pad[neighbor_pad==r] = 0

print(neighbor_attr[0])

neighbor_node = torch.cat([torch.full((batch_size, 1), model_.cls_token_e), neighbor_node], dim=1)
neighbor_attr = torch.cat([torch.full((batch_size, 1), model_.cls_token_r), neighbor_attr], dim=1)
neighbor_pad = torch.cat([torch.full((batch_size, 1), False), neighbor_pad], dim=1)

neighbor_node = model_.get_emb_e(neighbor_node)
neighbor_attr = model_.get_emb_rel(neighbor_attr)
neighbor_node.shape, neighbor_attr.shape, neighbor_pad.shape

tensor([18])
tensor([ 3,  7, 18,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,

(torch.Size([4, 484, 200]), torch.Size([4, 484, 200]), torch.Size([4, 484]))

In [56]:
# neighbor_node = model_.neighbor_node_linear(neighbor_node)
# neighbor_attr = model_.neighbor_edge_linear(neighbor_attr)
neighbor_node.shape, neighbor_attr.shape
neighbor_node = nn.Softmax(dim=2)(neighbor_node)
neighbor_attr = nn.Softmax(dim=2)(neighbor_attr)

In [58]:
x = neighbor_node + neighbor_attr
model_.transformer_encoder.forward(neighbor_node, src_key_padding_mask=neighbor_pad, mask=neighbor_pad).shape

RuntimeError: The shape of the 2D attn_mask is torch.Size([4, 484]), but should be (484, 484).

In [83]:
lr=1e-4
opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=args.l2)
for idx, (er, e2s) in enumerate(train):
    opt.zero_grad()
    er, e2s = er.to(device), e2s.to(device)
    e, r = er.split(1, 1)
    # e2s = ((1.0 - args.label_smoothing) * e2s) + (1.0 / e2s.size(1))
    # sum_train += (e2s[e2s != 0]).sum()

    pred: torch.Tensor = model.forward((e, r))
    _loss = model.loss(pred, e2s)
    _loss.backward()
    opt.step()
    # loss check
    _loss = _loss.detach().sum()
    # loss += _loss
    del e, r, er, pred, e2s, _loss
    torch.cuda.empty_cache()

KeyboardInterrupt: 

In [93]:
import matplotlib.pyplot as plt
tmp = torch.count_nonzero(model_.node2neighbor_pad==False, dim=1)
model_.node2neighbor_node[tmp>50].shape

torch.Size([101, 483])